<a href="https://colab.research.google.com/github/ZhangEnsure/pytorch-d2l-zh/blob/master/4.%E5%BE%AA%E7%8E%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文本预处理



In [ ]:
%pip install d2l
%pip install matplotlib_inline
%pip install matplotlib==3.0

In [2]:
import collections
import re
from d2l import torch as d2l

将数据集读取到由多条文本行组成的列表中

In [3]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt','090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(lines[0])
print(lines[10])

the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


每个文本序列又被拆分成一个词元列表

In [10]:
def tokenize(lines, token='word'):  
  """将文本行拆分为单词或字符词元"""
  if token == 'word':
    # line.split()返回一个列表，最外层还有一个列表
    # 所以这是一个二维的列表
    return [line.split() for line in lines]
  elif token == 'char':
    return [list(line) for line in lines]
  else:
    print('错误：未知词元类型：' + token)

# print(lines[0])
# the time machine by h g wells
# print(list(lines[0]))
# ['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'b', 'y', ' ', 'h', ' ', 'g', ' ', 'w', 'e', 'l', 'l', 's']

tokens = tokenize(lines, token='char')
for i in range(11):
  print(tokens[i])

['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'b', 'y', ' ', 'h', ' ', 'g', ' ', 'w', 'e', 'l', 'l', 's']
[]
[]
[]
[]
['i']
[]
[]
['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 't', 'r', 'a', 'v', 'e', 'l', 'l', 'e', 'r', ' ', 'f', 'o', 'r', ' ', 's', 'o', ' ', 'i', 't', ' ', 'w', 'i', 'l', 'l', ' ', 'b', 'e', ' ', 'c', 'o', 'n', 'v', 'e', 'n', 'i', 'e', 'n', 't', ' ', 't', 'o', ' ', 's', 'p', 'e', 'a', 'k', ' ', 'o', 'f', ' ', 'h', 'i', 'm']
['w', 'a', 's', ' ', 'e', 'x', 'p', 'o', 'u', 'n', 'd', 'i', 'n', 'g', ' ', 'a', ' ', 'r', 'e', 'c', 'o', 'n', 'd', 'i', 't', 'e', ' ', 'm', 'a', 't', 't', 'e', 'r', ' ', 't', 'o', ' ', 'u', 's', ' ', 'h', 'i', 's', ' ', 'g', 'r', 'e', 'y', ' ', 'e', 'y', 'e', 's', ' ', 's', 'h', 'o', 'n', 'e', ' ', 'a', 'n', 'd']
['t', 'w', 'i', 'n', 'k', 'l', 'e', 'd', ' ', 'a', 'n', 'd', ' ', 'h', 'i', 's', ' ', 'u', 's', 'u', 'a', 'l', 'l', 'y', ' ', 'p', 'a', 'l', 'e', ' ', 'f', 'a', 'c', 'e', ' ', 'w', 'a', 's', ' ', 'f

构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中

In [5]:
class Vocab:  
  """文本词表"""
  def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
    if tokens is None:
      tokens = []
    if reserved_tokens is None:
      reserved_tokens = []
    # 返回一个Counter对象，数据是字典。每一个词是键，出现的次数是值
    counter = count_corpus(tokens)
    # counter.items() is a dict_items
    # 我们将时光机器所有的词元统计出现的次数，并进行排序，按从大到小的顺序排列
    self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    self.idx_to_token = ['<unk>'] + reserved_tokens
    # 这是一个字典类型的数据，根据token键得到value（下标）
    # 因为最后代码处理的不是字符，而是顺序递增的数字
    self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
    for token, freq in self._token_freqs:
      if freq < min_freq:
        break
      if token not in self.token_to_idx:
        self.idx_to_token.append(token)
        self.token_to_idx[token] = len(self.idx_to_token) - 1

  def __len__(self):
    return len(self.idx_to_token)

  # 根据tokens返回idx
  def __getitem__(self, tokens):
    # tokens可能不只是一个字符，而是一个列表或者元组
    # 这里tokens是一个字符
    if not isinstance(tokens, (list, tuple)):
      return self.token_to_idx.get(tokens, self.unk)
    # 这里递归调用自己
    return [self.__getitem__(token) for token in tokens]

  def to_tokens(self, indices):
    if not isinstance(indices, (list, tuple)):
      return self.idx_to_token[indices]
    return [self.idx_to_token[index] for index in indices]

  @property
  def unk(self):
    return 0

  @property
  def token_freqs(self):
    return self._token_freqs

# 返回Counter({'the': 4, 'time': 1, 'machine': 1, 'by': 1, 'h': 1, 'g': 1, 'wells': 1})的数据
def count_corpus(tokens):  
  """统计词元的频率"""
  if len(tokens) == 0 or isinstance(tokens[0], list):
    tokens = [token for line in tokens for token in line]
  return collections.Counter(tokens)

In [6]:
# 简单测试一次这个count_corpus()函数
my_tokens = tokens[0]
# my_tokens.append('the')
# print(my_tokens)
# print(tokens[:10])
res = count_corpus(tokens[:10])
print(type(res.items()))
token_frequence = sorted(res.items(), key=lambda x: x[1], reverse=True)
print(token_frequence)
# print(['<unk>']+[])

<class 'dict_items'>
[('the', 2), ('time', 2), ('to', 2), ('machine', 1), ('by', 1), ('h', 1), ('g', 1), ('wells', 1), ('i', 1), ('traveller', 1), ('for', 1), ('so', 1), ('it', 1), ('will', 1), ('be', 1), ('convenient', 1), ('speak', 1), ('of', 1), ('him', 1), ('was', 1), ('expounding', 1), ('a', 1), ('recondite', 1), ('matter', 1), ('us', 1), ('his', 1), ('grey', 1), ('eyes', 1), ('shone', 1), ('and', 1)]


构建词表

In [7]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


将每一条文本行转换成一个数字索引列表

In [8]:
for i in [0, 10]:
  print(i)
  print('文本:', tokens[i])
  print('索引:', vocab[tokens[i]])

0
文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
10
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


将所有功能打包到`load_corpus_time_machine`函数中

In [12]:
def load_corpus_time_machine(max_tokens=-1):  
  """返回时光机器数据集的词元索引列表和词表"""
  lines = read_time_machine()
  tokens = tokenize(lines, 'char')
  vocab = Vocab(tokens)
  corpus = [vocab[token] for line in tokens for token in line]
  if max_tokens > 0:
      corpus = corpus[:max_tokens]
  return corpus, vocab

# 26个字母，加<UNK>，加空格
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)